### 1. إعداد البيئة وتحميل البيانات
نقوم باستيراد المكتبات اللازمة، وتحميل ملف البيانات (CSV)، ثم عرض نظرة عامة تشمل الأبعاد، العينات، والقيم المفقودة.

In [1]:
import pandas as pd

# ==========================================
# إعدادات الملف
# ==========================================
# ضع اسم ملف البيانات الخاص بك هنا بدقة
file_name = 'dataset/AHQAD.csv' 

try:
    # قراءة الملف (نستخدم utf-8-sig لأنه الأفضل مع اللغة العربية وملفات Excel)
    df = pd.read_csv(file_name, encoding='utf-8-sig')
    
    print("✅ تم تحميل الملف بنجاح!\n")
    print("-" * 50)

    # 1. نظرة عامة سريعة
    print(f"📊 أبعاد البيانات (عدد الصفوف, عدد الأعمدة): {df.shape}")
    print("-" * 50)

    # 2. عرض أول 5 صفوف (للتأكد من شكل البيانات)
    print("👀 عينة من البيانات (أول 5 صفوف):")
    display(df.head()) # دالة display تعمل بشكل جميل داخل Jupyter
    print("-" * 50)

    # 3. فحص أسماء الأعمدة
    print("📋 أسماء الأعمدة الموجودة:")
    for col in df.columns:
        print(f"   - {col}")
    print("-" * 50)

    # 4. فحص القيم الفارغة (Null Values)
    print("search🔍 فحص الخلايا الفارغة (Missing Values):")
    null_counts = df.isnull().sum()
    if null_counts.sum() == 0:
        print("   ✅ لا توجد أي قيم مفقودة.")
    else:
        print(null_counts[null_counts > 0])
    print("-" * 50)

    # 5. فحص التكرارات (Duplicates)
    duplicates_count = df.duplicated().sum()
    print(f"🔄 فحص الصفوف المكررة بالكامل:")
    if duplicates_count == 0:
        print("   ✅ لا توجد صفوف مكررة.")
    else:
        print(f"   ⚠️ تم العثور على {duplicates_count} صف مكرر!")
        # عرض مثال على التكرار إن وجد
        print("   مثال على البيانات المكررة:")
        display(df[df.duplicated()].head(3))
        
    print("-" * 50)
    print("نهاية التقرير.")

except FileNotFoundError:
    print(f"❌ خطأ: لم يتم العثور على الملف باسم '{file_name}'.")
    print("تأكد من وضع ملف الـ csv في نفس مجلد الجوبيتر أو كتابة المسار الصحيح.")
except Exception as e:
    print(f"❌ حدث خطأ غير متوقع: {e}")

✅ تم تحميل الملف بنجاح!

--------------------------------------------------
📊 أبعاد البيانات (عدد الصفوف, عدد الأعمدة): (808472, 4)
--------------------------------------------------
👀 عينة من البيانات (أول 5 صفوف):


,Unnamed: 0,Question,Answer,Category
0,0,اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز...,المهم بالموضوع ما هي درجة ال BIRADS الموصوفة ل...,جراحة عامة
1,1,انا اعاني من اشياء تجبرني على فعل امور لا اريد...,الف سلامه عليك ، انت في حاجه الي معالج نفسي لم...,أمراض نفسية
2,2,أريد الاستفسار عن مضاعفات عملية استئصال ورم ال...,كغيرها من العمليات الجراحية ترتبط عملية استئصا...,أمراض الغدد الصماء
3,3,اليوم حاسس ان عندى حاجه فى الدبر من جوا مش عاي...,يغلب ان تعكس الحالة بواسير شرجية داخلية من الد...,جراحة عامة
4,4,ليه فجأة بحس انى مش عارفة اتنفس وجسمى بينمل وي...,من المحتمل عضوي او تاثير جانبي من اي ادويه بيت...,أمراض نفسية


--------------------------------------------------
📋 أسماء الأعمدة الموجودة:
   - Unnamed: 0
   - Question
   - Answer
   - Category
--------------------------------------------------
search🔍 فحص الخلايا الفارغة (Missing Values):
Question    1
dtype: int64
--------------------------------------------------
🔄 فحص الصفوف المكررة بالكامل:
   ✅ لا توجد صفوف مكررة.
--------------------------------------------------
نهاية التقرير.


### 2. معالجة وتنظيف النصوص
تتضمن هذه المرحلة إزالة التكرارات والقيم الفارغة، ثم تطبيق دالة مخصصة لتنظيف النصوص العربية (إزالة التشكيل، توحيد الأحرف، إزالة الرموز)، وأخيراً حفظ النتائج.

In [2]:
import pandas as pd
import re

# تحميل الملف مرة أخرى (أو استخدام المتغير df من الخلية السابقة)
file_name = 'dataset/AHQAD.csv'
try:
    df = pd.read_csv(file_name, encoding='utf-8-sig')
except Exception as e:
    # Fallback to cleaned if original not found during re-run
    try: 
        df = pd.read_csv('medical_data_cleaned.csv', encoding='utf-8-sig')
    except: 
        print("File not found, please check path")

# ==========================================
# 1. إصلاح مشاكل الهيكلية (Missing + Unnamed)
# ==========================================

# حذف عمود الفهرس الزائد إذا وجد
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
    print("✅ تم حذف عمود 'Unnamed: 0' بنجاح.")

# حذف أي صف يحتوي على قيمة فارغة في السؤال أو الجواب
initial_count = len(df)
df = df.dropna(subset=['Question', 'Answer'])
final_count = len(df)
print(f"✅ تم حذف {initial_count - final_count} صفوف فارغة.")

# ==========================================
# 2. تحليل التصنيفات (Categories) - طلبك الخاص
# ==========================================
print("\n📊 توزيع البيانات حسب التصنيف (Category):")
print("-" * 40)
if 'Category' in df.columns:
    category_counts = df['Category'].value_counts()
    print(category_counts)
else:
    print("⚠️ عمود التصنيف غير موجود.")
print("-" * 40)

# ==========================================
# 3. تنظيف النصوص (Preprocessing)
# ==========================================
print("\n🧹 جاري تنظيف النصوص (سؤال وجواب)...")

# Pre-compile regex patterns for efficiency
p_tashkeel = re.compile(r'[\u064B-\u065F\u0670]')
p_tatweel = re.compile(r'[\u0640]')
p_alef = re.compile(r'[أإآ]')
p_non_alphanum = re.compile(r'[^\w\s]')
p_spaces = re.compile(r'\s+')

def clean_arabic_text(text):
    if not isinstance(text, str):
        return ""
    
    # 1. إزالة التشكيل
    text = p_tashkeel.sub('', text)
    # 2. إزالة التطويل (ـ)
    text = p_tatweel.sub('', text)
    # 3. توحيد الألف (أ، إ، آ -> ا)
    text = p_alef.sub('ا', text)
    # 4. توحيد الياء المتطرفة (ى -> ي)
    text = text.replace('ى', 'ي')
    # 5. إزالة الرموز غير الضرورية
    text = p_non_alphanum.sub(' ', text)
    # 6. إزالة المسافات الزائدة
    text = p_spaces.sub(' ', text).strip()
    
    return text

# تطبيق التنظيف على الأعمدة
df['cleaned_question'] = df['Question'].apply(clean_arabic_text)
df['cleaned_answer'] = df['Answer'].apply(clean_arabic_text)

# عرض مثال للتأكد
print("\n📝 مثال قبل وبعد التنظيف:")
print(f"قبل: {df.iloc[0]['Question']}")
print(f"بعد: {df.iloc[0]['cleaned_question']}")

# ==========================================
# 4. الحفظ النهائي
# ==========================================
# حفظ نسخة CSV نظيفة (للتوثيق أو الاستخدام لاحقاً)
df.to_csv('medical_data_cleaned.csv', index=False, encoding='utf-8-sig')

# حفظ نسخة نصية (TXT) جاهزة لتدريب GPT
# التنسيق: User: [Question] \n Assistant: [Answer] <|endoftext|>
def format_gpt(row):
    return f"Question: {row['cleaned_question']}\nAnswer: {row['cleaned_answer']}<|endoftext|>\n"

with open('train_data.txt', 'w', encoding='utf-8') as f:
    # نكتب كل الصفوف في ملف واحد
    for text in df.apply(format_gpt, axis=1):
        f.write(text)

print("\n💾 تم الحفظ بنجاح!")
print("1. ملف البيانات النظيف: medical_data_cleaned.csv")
print("2. ملف التدريب النصي: train_data.txt (هذا الملف الذي سنستخدمه في التدريب)")

✅ تم حذف عمود 'Unnamed: 0' بنجاح.
✅ تم حذف 1 صفوف فارغة.

📊 توزيع البيانات حسب التصنيف (Category):
----------------------------------------
Category
أمراض نسائية                        162141
الصحة الجنسية                        96398
أمراض العضلات والعظام و المفاصل      45826
أمراض المسالك البولية والتناسلية     42599
الأمراض الجلدية                      41760
                                     ...  
علم النبات                               5
علم الأنسجة                              5
أخبار الطبي                              2
كيمياء عضوية                             1
علم البيئة                               1
Name: count, Length: 90, dtype: int64
----------------------------------------

🧹 جاري تنظيف النصوص (سؤال وجواب)...

📝 مثال قبل وبعد التنظيف:
قبل: اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز غير كتليه تظهر تعزيز باكر يتراجع لاحقا تقيس ٢ب٣ سم
مامعنى هل كلام ؟
اخبرني الطبيب بانها اجتهاد من طبيبه الاشعه ولا...
بعد: اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز غير كتليه تظهر

### 3. التحقق من جودة التنظيف
نستعرض عينات من الملفات المحفوظة (سواء CSV أو ملف التدريب النصي) لضمان أن البيانات جاهزة للمرحلة التالية.

In [1]:
import pandas as pd

# ==========================================
# 1. عرض عينة من ملف CSV النظيف
# ==========================================
print("📊 1. عينة من ملف البيانات النظيف (medical_data_cleaned.csv):")
try:
    df_clean = pd.read_csv('medical_data_cleaned.csv', encoding='utf-8-sig')
    display(df_clean.head()) # يعرض أول 5 صفوف بتنسيق جدول
    print(f"عدد الصفوف الكلي: {len(df_clean)}")
except FileNotFoundError:
    print("❌ الملف medical_data_cleaned.csv غير موجود.")

print("-" * 50)

# ==========================================
# 2. عرض عينة من ملف التدريب النصي
# ==========================================
print("\n📝 2. عينة من ملف التدريب (train_data.txt) - كيف يراه النموذج:")
try:
    with open('train_data.txt', 'r', encoding='utf-8') as f:
        # سنقرأ أول 500 حرف فقط لنرى الشكل العام
        content = f.read(500)
        print(content)
        print("\n... (بقية الملف)")
except FileNotFoundError:
    print("❌ الملف train_data.txt غير موجود.")

📊 1. عينة من ملف البيانات النظيف (medical_data_cleaned.csv):


,Question,Answer,Category,cleaned_question,cleaned_answer
0,اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز...,المهم بالموضوع ما هي درجة ال BIRADS الموصوفة ل...,جراحة عامة,اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز...,المهم بالموضوع ما هي درجة ال BIRADS الموصوفة ل...
1,انا اعاني من اشياء تجبرني على فعل امور لا اريد...,الف سلامه عليك ، انت في حاجه الي معالج نفسي لم...,أمراض نفسية,انا اعاني من اشياء تجبرني علي فعل امور لا اريد...,الف سلامه عليك انت في حاجه الي معالج نفسي لمسا...
2,أريد الاستفسار عن مضاعفات عملية استئصال ورم ال...,كغيرها من العمليات الجراحية ترتبط عملية استئصا...,أمراض الغدد الصماء,اريد الاستفسار عن مضاعفات عملية استئصال ورم ال...,كغيرها من العمليات الجراحية ترتبط عملية استئصا...
3,اليوم حاسس ان عندى حاجه فى الدبر من جوا مش عاي...,يغلب ان تعكس الحالة بواسير شرجية داخلية من الد...,جراحة عامة,اليوم حاسس ان عندي حاجه في الدبر من جوا مش عاي...,يغلب ان تعكس الحالة بواسير شرجية داخلية من الد...
4,ليه فجأة بحس انى مش عارفة اتنفس وجسمى بينمل وي...,من المحتمل عضوي او تاثير جانبي من اي ادويه بيت...,أمراض نفسية,ليه فجاة بحس اني مش عارفة اتنفس وجسمي بينمل وي...,من المحتمل عضوي او تاثير جانبي من اي ادويه بيت...


عدد الصفوف الكلي: 808471
--------------------------------------------------

📝 2. عينة من ملف التدريب (train_data.txt) - كيف يراه النموذج:
Question: اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز غير كتليه تظهر تعزيز باكر يتراجع لاحقا تقيس ٢ب٣ سم مامعني هل كلام اخبرني الطبيب بانها اجتهاد من طبيبه الاشعه ولا
Answer: المهم بالموضوع ما هي درجة ال BIRADS الموصوفة لتحديد خطورة الموصوف<|endoftext|>
Question: انا اعاني من اشياء تجبرني علي فعل امور لا اريدها واذا لم اعطها اهتمام تزيد وياتي تورتر شديد مع ضيق وصور في الذهن محرمه فالدين وتزعجني كثيرا اعاني سنه احتاج
Answer: الف سلامه عليك انت في حاجه الي معالج نفسي لمساعدتك<|endoftext|>
Quest

... (بقية الملف)


### 4. فحص النصوص الطويلة
تعديل إعدادات العرض للتمكن من قراءة النصوص الكاملة (بدون اختصار) للتأكد من سلامة المعنى بعد المعالجة.

In [2]:
import pandas as pd

# =========================================================
# 🛠️ إعدادات لفك القيود عن العرض (الحل السحري)
# =========================================================
# هذا السطر يجعل العمود يتسع لأي كمية من النصوص دون قصها
pd.set_option('display.max_colwidth', None)

# هذا السطر يعرض جميع الأعمدة (إذا كان لديك أعمدة كثيرة)
pd.set_option('display.max_columns', None)

# =========================================================
# 1. عرض عينة من ملف CSV (كاملة)
# =========================================================
print("📊 عرض عينة من البيانات (النص الكامل):")
try:
    df_clean = pd.read_csv('medical_data_cleaned.csv', encoding='utf-8-sig')
    
    # سنعرض أول 3 صفوف فقط لكي لا تمتلئ الشاشة، ولكن النص سيكون كاملاً
    display(df_clean[['cleaned_question', 'cleaned_answer']].head(3))
    
except FileNotFoundError:
    print("❌ الملف غير موجود.")

print("-" * 50)

# =========================================================
# 2. طريقة أخرى: طباعة نص واحد فقط كـ "نص خام" (للتأكد 100%)
# =========================================================
print("\n📝 عرض مثال واحد كنص عادي (بدون جدول):")
if 'df_clean' in locals():
    example_q = df_clean.iloc[0]['cleaned_question']
    example_a = df_clean.iloc[0]['cleaned_answer']
    
    print(f"🔹 السؤال الكامل:\n{example_q}")
    print(f"\n🔹 الجواب الكامل:\n{example_a}")

📊 عرض عينة من البيانات (النص الكامل):


,cleaned_question,cleaned_answer
0,اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز غير كتليه تظهر تعزيز باكر يتراجع لاحقا تقيس ٢ب٣ سم مامعني هل كلام اخبرني الطبيب بانها اجتهاد من طبيبه الاشعه ولا,المهم بالموضوع ما هي درجة ال BIRADS الموصوفة لتحديد خطورة الموصوف
1,انا اعاني من اشياء تجبرني علي فعل امور لا اريدها واذا لم اعطها اهتمام تزيد وياتي تورتر شديد مع ضيق وصور في الذهن محرمه فالدين وتزعجني كثيرا اعاني سنه احتاج,الف سلامه عليك انت في حاجه الي معالج نفسي لمساعدتك
2,اريد الاستفسار عن مضاعفات عملية استئصال ورم الدماغ هل عملية استئصال ورم الدماغ الحميد خطير,كغيرها من العمليات الجراحية ترتبط عملية استئصال ورم الدماغ الحميد او الخبيث بالعديد من المضاعفات المختلفة والتي يجب ان يتم مناقشتها مع الطبيب المختص قبل اجراء العملية ومناقشة خيارات ادارة الحالة في حال حدوثها ومن ابرز المضاعفات التي ترافق عملية استئصال ورم الدماغ الحميد ما يلي انتفاخ او تورم الدماغ او ما تسمي الوذمة الدماغية حيث يزداد حجم الدماغ نتيجة تجمع السوائل بداخله وتعد هذه المضاعفات حالة طبية طارئة تستدعي التدخل باسرع وقت ممكن لتجنب اي مضاعفات اخري ويمكن ملاحظة ذلك من خلال اعراض الصداع والضعف والدوار والتغيرات في السلوك والاضطرابات في التوازن والاضطرابات والنوبات وعدم وضوح الرؤية وغيرها حدوث نزيف في الدماغ زيادة خطر حدوث النوبات حدوث تلف في الدماغ وفقدان في وظائف الدماغ المختلفة ويجب مراقبة المريض بصورة دقيقة وملاحظة اي تغيرات حيث قد تحدث هذه الاثار مباشرة بعد العملية او بعد عدة ايام وحتي عدة اسابيع كذلك شعور المريض بالضعف والدوخة مباشرة بعد العملية حدوث اضطرابات في عملية النطق زيادة خطر الاصابة بالالتهاب والعدوي بعد العملية الجراحية زيادة خطر الجلطات الدموية حدوث اضطرابات في الذاكرة حدوث اضطرابات في الوظيفة الحركية في الجسم للمزيد عملية استئصال الورم من الدماغ كم تستغرق وما هي نسبة نجاحها 8 من ابرز اعراض ورم الدماغ


--------------------------------------------------

📝 عرض مثال واحد كنص عادي (بدون جدول):
🔹 السؤال الكامل:
اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز غير كتليه تظهر تعزيز باكر يتراجع لاحقا تقيس ٢ب٣ سم مامعني هل كلام اخبرني الطبيب بانها اجتهاد من طبيبه الاشعه ولا

🔹 الجواب الكامل:
المهم بالموضوع ما هي درجة ال BIRADS الموصوفة لتحديد خطورة الموصوف
